In [1]:
import pandas as pd
pd.set_option('display.max_colwidth',100)
import nltk

In [5]:
t_data=pd.read_csv('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/malyalam/malayalam_hope_train.csv',names=["text","label","NaN"], sep='\t')
t_data.pop('NaN')
t_data.head(9)

,text,label
0,@arya s nair may be athile karthikayude charctr bisexual ayirikkum,Non_hope_speech
1,വാങ്ങിയത് എന്ത് കുന്തം ആയാലും കളയാൻ പറ്റില്ലല്ലോ ഇനി വാങ്ങാതിരിക്കാൻ ശ്രദ്ധിക്കുക Jai India,Non_hope_speech
2,മാറുമറയ്ക്കാൻ നടത്തിയ സമരം ഒരു previlege issue ആയിരുന്നു...അത് ഈ വിഷയവുമായി കമ്പയർ ചെയ്യാൻ പറ്റ...,Hope_speech
3,ഇഷാനെ കൂടുതല് കെട്ടിപിടിക്കേണ്ട ഞങ്ങൾക്കറിയാം ഇഷാന് സൂര്യയോട് ഇച്ചിരി കൊഞ്ചൽ കൂടുതലാണെന്ന്,Non_hope_speech
4,ഇന്ത്യ ഇസ്‌ലാമിക രാജ്യമല്ല.. ഇന്ത്യൻ ഭരണഘടന മാത്രമേ ഇന്ത്യൻ പൗരന്മാർ നോക്കേണ്ടതുളൂ. ഇന്ത്യൻ നിയ...,Hope_speech
5,Xenderin pakaram okke entha ubayokhikkuka,Non_hope_speech
6,ചൈനയുടെ കാര്യം അവര് നോക്കട്ടെ.. nഇന്ത്യയിൽ പെട്രോളിന് ദിവസവും വിലക്കുട്ടി കൊണ്ടിരിക്കുകയാണ് അതിന...,Non_hope_speech
7,@Akhil Dev സോറി ഇപ്പഴാ മനസിലായത്,Non_hope_speech
8,നിങ്ങൾക് നല്ലത് വരും.....,Hope_speech


In [6]:
t_data['text']=t_data['text'].str.lower()       #changing into lower case       (remove and check acc too)

In [7]:
t_data['text']=t_data['text'].str.strip()       #remove white spaces
t_data['text']=t_data['text'].str.replace(r'\d+','')   #remove numbers
#t_data['text']=t_data['text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))  #removing emoji
t_data['text']=t_data['text'].str.replace('[^\w\s]','')      #removing punct

In [8]:
#removing url if any
import re
def remove_URL(txt):
    url= re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",txt)
t_data['text']=t_data['text'].apply(lambda x:remove_URL(x))

In [9]:
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
labelEncode=preprocessing.LabelEncoder()
labelEncode.fit(t_data['label'])
print (labelEncode.classes_)
train_labelEncode=labelEncode.transform(t_data['label'])
label=to_categorical(np.asarray(train_labelEncode))
label

['Hope_speech' 'Non_hope_speech' 'not-malayalam']


array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [10]:
#counting labels      
 
t_data['label'].value_counts()
#must use something to balance data

Non_hope_speech    6205
Hope_speech        1668
not-malayalam       691
Name: label, dtype: int64

In [11]:
#words in each line
 
t_data['totalwords'] = t_data['text'].str.count(' ') + 1
t_data

,text,label,totalwords
0,arya s nair may be athile karthikayude charctr bisexual ayirikkum,Non_hope_speech,10
1,വങങയത എനത കനത ആയല കളയൻ പററലലലല ഇന വങങതരകകൻ ശരദധകകക jai india,Non_hope_speech,11
2,മറമറയകകൻ നടതതയ സമര ഒര previlege issue ആയരനനഅത ഈ വഷയവമയ കമപയർ ചയയൻ പററലലതങകൾ പറഞഞത തററണസധരണ വഷ അ...,Hope_speech,28
3,ഇഷന കടതല കടടപടകകണട ഞങങൾകകറയ ഇഷന സരയയട ഇചചര കഞചൽ കടതലണനന,Non_hope_speech,10
4,ഇനതയ ഇസലമക രജയമലല ഇനതയൻ ഭരണഘടന മതരമ ഇനതയൻ പരനമർ നകകണടതള ഇനതയൻ നയമ പരകര പരയപർതതയയ രണടപർകക പരസപര ...,Hope_speech,22
...,...,...,...
8559,evan eatha eee pottan oru vivaravumilla,Non_hope_speech,6
8560,ithinu pakaramayi upayogikkan pattunna indian app undo,Non_hope_speech,7
8561,സറന നലലത വരടട,Non_hope_speech,3
8562,ഇങങനതത നലല നലല അറവകൾ പറഞഞ തരനന ഈ ഡകടർക ഇരകകടട ഒര നനദ,Hope_speech,11


In [12]:
import keras
from keras.preprocessing.text import Tokenizer
tok1 = Tokenizer(char_level=False, filters ='!"$%&@()*+,-./:;”“<=>?[\\]^_`{|}~\t\n', lower = True)

In [13]:
tok2=Tokenizer(char_level=True,filters ='!"$%&@()*+,-./:;”“<=>?[\\]^_`{|}~\t\n',lower=True)

In [14]:
tok1.fit_on_texts(t_data['text'])
words=len(tok1.word_counts)
words

28996

In [15]:
tok2.fit_on_texts(t_data['text'])

In [16]:
Charlist=tok2.word_index
#Charlist

In [17]:
wordlist=tok1.word_index
#wordlist

In [18]:
encode=tok1.texts_to_sequences(t_data['text'])
print(t_data['text'][5555])
encode[5555]

dr ന പല തവണ വളചച പരജയ പടട ഒര വകത ആണ


[39, 24, 11, 1306, 528, 2623, 854, 1, 917, 2]

In [43]:
##         Padding encoded sequence of words
from keras.preprocessing import sequence
max_length=20
padd = sequence.pad_sequences(encode, maxlen=max_length, padding='post')
padd

array([[ 3584,   227,  1049, ...,     0,     0,     0],
       [ 8672,    18,  2252, ...,     0,     0,     0],
       [ 8676,     5,  3586, ...,  8684,     9,    64],
       ...,
       [ 4712,   180,  1137, ...,     0,     0,     0],
       [ 5506,    31,    31, ...,     0,     0,     0],
       [   28,    68, 28994, ...,  8433,     0,     0]], dtype=int32)

In [44]:
# importing libraries for creating model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding,MaxPool1D
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D,LSTM,Bidirectional

#Embedding layer

In [45]:
Embedding_Layer=Embedding(input_dim = words + 1, output_dim = 300,input_length=max_length)

#CNN

In [46]:
model = Sequential()
model.add(Embedding_Layer)
model.add(Conv1D(filters =256,kernel_size=2, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Conv1D(filters =512,kernel_size=3, activation='relu'))
model.add(MaxPooling1D(3,padding='same'))
model.add(Flatten())
model.add(Dense(32, activation = 'relu'))
#model.add(Dense(4, activation = 'relu'))
model.add(Dense(3, activation = 'sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 300)           8699100   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 19, 256)           153856    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 9, 256)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 7, 512)            393728    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 3, 512)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1536)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)               

#LSTM

In [47]:
model_1 = Sequential()
model_1.add(Embedding_Layer)
model_1.add(LSTM(256,return_sequences=False))

model_1.add(Flatten())
model_1.add(Dense(32, activation = 'relu'))
#model.add(Dense(4, activation = 'relu'))
model_1.add(Dense(3, activation = 'sigmoid'))
model_1.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 300)           8699100   
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               570368    
_________________________________________________________________
flatten_4 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                8224      
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 99        
Total params: 9,277,791
Trainable params: 9,277,791
Non-trainable params: 0
_________________________________________________________________


#Bidirectional

In [48]:
model_2 = Sequential()
model_2.add(Embedding_Layer)
model_2.add(Bidirectional(LSTM(256,return_sequences=False)))

model_2.add(Flatten())
model_2.add(Dense(32, activation = 'relu'))
#model.add(Dense(4, activation = 'relu'))
model_2.add(Dense(3, activation = 'sigmoid'))
model_2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 300)           8699100   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               1140736   
_________________________________________________________________
flatten_5 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                16416     
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 99        
Total params: 9,856,351
Trainable params: 9,856,351
Non-trainable params: 0
_________________________________________________________________


#compile

In [49]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

#Training models

In [50]:
model.fit(padd,label,epochs=20,verbose=1,batch_size=64)

Epoch 1/20
134/134 [==============================] - 26s 191ms/step - loss: 0.4462 - acc: 0.7168
Epoch 2/20
134/134 [==============================] - 26s 192ms/step - loss: 0.1718 - acc: 0.9064
Epoch 3/20
134/134 [==============================] - 26s 193ms/step - loss: 0.0404 - acc: 0.9824
Epoch 4/20
134/134 [==============================] - 26s 191ms/step - loss: 0.0166 - acc: 0.9945
Epoch 5/20
134/134 [==============================] - 26s 192ms/step - loss: 0.0124 - acc: 0.9947
Epoch 6/20
134/134 [==============================] - 26s 192ms/step - loss: 0.0085 - acc: 0.9968
Epoch 7/20
134/134 [==============================] - 26s 193ms/step - loss: 0.0092 - acc: 0.9946
Epoch 8/20
134/134 [==============================] - 26s 191ms/step - loss: 0.0082 - acc: 0.9939
Epoch 9/20
134/134 [==============================] - 26s 191ms/step - loss: 0.0050 - acc: 0.9971
Epoch 10/20
134/134 [==============================] - 25s 189ms/step - loss: 0.0045 - acc: 0.9973
Epoch 11/20
134/134

In [51]:
model_1.fit(padd,label,epochs=20,verbose=1,batch_size=64)

Epoch 1/20
134/134 [==============================] - 32s 221ms/step - loss: 0.3386 - acc: 0.8122
Epoch 2/20
134/134 [==============================] - 29s 218ms/step - loss: 0.0364 - acc: 0.9840
Epoch 3/20
134/134 [==============================] - 29s 217ms/step - loss: 0.0198 - acc: 0.9912
Epoch 4/20
134/134 [==============================] - 29s 218ms/step - loss: 0.0127 - acc: 0.9937
Epoch 5/20
134/134 [==============================] - 29s 217ms/step - loss: 0.0116 - acc: 0.9943
Epoch 6/20
134/134 [==============================] - 30s 221ms/step - loss: 0.0103 - acc: 0.9953
Epoch 7/20
134/134 [==============================] - 30s 226ms/step - loss: 0.0112 - acc: 0.9950
Epoch 8/20
134/134 [==============================] - 30s 223ms/step - loss: 0.0126 - acc: 0.9948
Epoch 9/20
134/134 [==============================] - 30s 222ms/step - loss: 0.0081 - acc: 0.9956
Epoch 10/20
134/134 [==============================] - 30s 223ms/step - loss: 0.0105 - acc: 0.9948
Epoch 11/20
134/134

In [52]:
model_2.fit(padd,label,epochs=20,verbose=1,batch_size=64)

Epoch 1/20
134/134 [==============================] - 52s 365ms/step - loss: 0.2254 - acc: 0.8856
Epoch 2/20
134/134 [==============================] - 49s 364ms/step - loss: 0.0126 - acc: 0.9928
Epoch 3/20
134/134 [==============================] - 49s 365ms/step - loss: 0.0068 - acc: 0.9967
Epoch 4/20
134/134 [==============================] - 49s 363ms/step - loss: 0.0047 - acc: 0.9969
Epoch 5/20
134/134 [==============================] - 49s 363ms/step - loss: 0.0059 - acc: 0.9959
Epoch 6/20
134/134 [==============================] - 48s 361ms/step - loss: 0.0069 - acc: 0.9957
Epoch 7/20
134/134 [==============================] - 47s 351ms/step - loss: 0.0060 - acc: 0.9963
Epoch 8/20
134/134 [==============================] - 48s 358ms/step - loss: 0.0042 - acc: 0.9969
Epoch 9/20
134/134 [==============================] - 47s 350ms/step - loss: 0.0035 - acc: 0.9970
Epoch 10/20
134/134 [==============================] - 48s 355ms/step - loss: 0.0040 - acc: 0.9969
Epoch 11/20
134/134

#Development data

In [53]:
d_data=pd.read_csv('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/malyalam/malayalam_hope_dev.csv',names=['text','label','nan'],sep='\t')

In [54]:
d_data=d_data[["text","label"]]
d_data

,text,label
0,ജനകീയ കോടതിയുടെ വളരെ മികച്ച എപിസോടുകളിൽ ഒന്ന്... നല്ല കലക്കൻ സംവാദം... കിടു ചർച്ച... nനിലപാടുകൾn...,Hope_speech
1,Opera Mini - യുടെ അത്രേം വരില്ല UC,Non_hope_speech
2,മറ്റുള്ള hetero sexuals നെ പോലെ കഴിവും സംസ്ക്കാരവുമൊക്കെയുള്ള വിഭാഗമാണവരും. അവരുടെ കുറവുകൾ മനസ്സ...,Hope_speech
3,@Mr President നിനക്ക് ഓക്കെ അത നല്ലത്...,Non_hope_speech
4,RSS തീവ്രവാദികൾ നേരത്തെ പദ്ധതി ഇട്ട പ്രകാരം ചൈനയുമായി ചേർന്ന് നടത്തുന്ന ഒരു നാടകമാണിത്. ബലിയാടാവ...,Non_hope_speech
...,...,...
1065,വാട്സ്ആപ്പ് ന് പകരം telegram app ഇത് ഇന്ത്യ യുടെ ആണ്,Non_hope_speech
1066,അടിസ്ഥാനമായി പള്ളിപ്രമാണങ്ങൾ മാറ്റിയെഴുതേണ്ടി വരും. nnഅരുൺ പൊളിയെ,Non_hope_speech
1067,Vella samsarikumpo blu ne nok..chevi kelkaatha pennine pole,Non_hope_speech
1068,നികേഷിന്റെ വോളിൽ അമമയുടെ കൂടെ രണ്ടു പേരും നിൽക്കുന്ന ഫോട്ടോ ഉണ്ട്,Non_hope_speech


In [55]:
#processing as training set

d_data['text']=d_data['text'].str.lower()       #changing into lower case       (remove and check acc too)
d_data['text']=d_data['text'].str.strip()       #remove white spaces
d_data['text']=d_data['text'].str.replace(r'\d+','')   #remove numbers
#d_data['text']=d_data['text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))  #removing emoji
d_data['text']=d_data['text'].str.replace('[^\w\s]','')      #removing punct
d_data['text']=d_data['text'].apply(lambda x:remove_URL(x))
#d_data['text'] = d_data['text'].apply(lambda x: remove_sw(x))


In [56]:
encoded1 =tok1.texts_to_sequences(d_data['text'])
print(d_data['text'][0])
encoded1[0]

ജനകയ കടതയട വളര മകചച എപസടകളൽ ഒനന നലല കലകകൻ സവദ കട ചർചച nനലപടകൾnപതയ കഴചപപടകൾ ndr arun


[1267, 65, 1612, 14, 31, 21995, 40, 977, 18818, 110]

In [57]:
padded = sequence.pad_sequences(encoded1, maxlen=max_length, padding='post')
padded[0]

array([ 1267,    65,  1612,    14,    31, 21995,    40,   977, 18818,
         110,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0], dtype=int32)

In [58]:
val_labelEncode=labelEncode.transform(d_data['label'])
val_label=to_categorical(np.asarray(val_labelEncode))
val_label[0]

array([1., 0., 0.], dtype=float32)

#prediction

In [59]:
# devlopment prediction of CNN

dev_predictions = model.predict(padded)
# Finding out of two output neurons which one has highest probability
# It will return us the predicted class
# It will convert probability into final class 1 and 0
dev_predictions1 = np.zeros_like(dev_predictions)
dev_predictions1[np.arange(len(dev_predictions)), dev_predictions.argmax(1)] = 1
#print(dev_predictions)
#print(dev_predictions1)

In [60]:
# devlopment prediction of Lstm

l_dev_predictions = model_1.predict(padded)
# Finding out of two output neurons which one has highest probability
# It will return us the predicted class
# It will convert probability into final class 1 and 0
l_dev_predictions1 = np.zeros_like(l_dev_predictions)
l_dev_predictions1[np.arange(len(l_dev_predictions)), l_dev_predictions.argmax(1)] = 1
#print(l_dev_predictions)
#print(l_dev_predictions1)

In [61]:
# devlopment prediction of BiLSTM

b_dev_predictions = model_2.predict(padded)
# Finding out of two output neurons which one has highest probability
# It will return us the predicted class
# It will convert probability into final class 1 and 0
b_dev_predictions1 = np.zeros_like(b_dev_predictions)
b_dev_predictions1[np.arange(len(b_dev_predictions)), b_dev_predictions.argmax(1)] = 1
#print(b_dev_predictions)
#print(b_dev_predictions1)

#Acuracy report

In [62]:
#Accuracy CNN
from sklearn.metrics import classification_report
print(classification_report(val_label,dev_predictions1))

              precision    recall  f1-score   support

           0       0.48      0.50      0.49       190
           1       0.85      0.81      0.83       784
           2       0.54      0.67      0.60        96

   micro avg       0.74      0.74      0.74      1070
   macro avg       0.62      0.66      0.64      1070
weighted avg       0.75      0.74      0.75      1070
 samples avg       0.74      0.74      0.74      1070



In [63]:
#Accuracy Lstm
print(classification_report(val_label,l_dev_predictions1))

              precision    recall  f1-score   support

           0       0.52      0.53      0.53       190
           1       0.86      0.81      0.83       784
           2       0.47      0.68      0.56        96

   micro avg       0.75      0.75      0.75      1070
   macro avg       0.62      0.67      0.64      1070
weighted avg       0.76      0.75      0.75      1070
 samples avg       0.75      0.75      0.75      1070



In [64]:
#Accuracy BiiLstm
print(classification_report(val_label,b_dev_predictions1))

              precision    recall  f1-score   support

           0       0.55      0.53      0.54       190
           1       0.86      0.81      0.84       784
           2       0.47      0.73      0.57        96

   micro avg       0.76      0.76      0.76      1070
   macro avg       0.63      0.69      0.65      1070
weighted avg       0.77      0.76      0.76      1070
 samples avg       0.76      0.76      0.76      1070



#Testing Data

In [65]:
t_data=pd.read_csv('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/malyalam/malayalam_hope_test.csv',names=['text'],sep=',')
t_data

,text
0,അതെ അണപൊട്ടി ഒഴുകുകയാണ്
1,കെവിൻ avde pettupoyi🤣🤣🤣
2,@my voice ullup illa daaa tayoli ath kond alle ninte ammaye njan panniyatha monee
3,@Albin Baiju
4,I respect you..
...,...
1066,ചേട്ടാ അതിന് ഇവിടെ ആരാണ് ഇന്ത്യൻ മൊബൈൽ റിവ്യൂ ചെയ്യുന്നത് ചില ഉത്തരേന്ത്യൻ ചാനൽ ചെയ്യും ഇവിടെ ...
1067,@surya ishaan
1068,ഏതാണ് ആ മൂലയിൽ ഇരിക്കുന്ന അപ്പൂപ്പൻ
1069,ഏതാ ആ ചൊമന്ന ടി ഷർട്ടിന് മേൽ കോട്ടിട്ട വവ്വാൽ...
